In [6]:
import pandas as pd
import pyodbc

# Getting average cost for each ActivityCode from myANALYTIC --> df3
sql_query = """
	SELECT
		ILART_MaintenanceActivityTypeID,
		AVG(ActualCost) AS 'AverageCost'
	FROM (
		SELECT
			ILART_MaintenanceActivityTypeID,
			ActualCost
		FROM [myANALYTICS_SP].[silver.fact.enterprise.asset].[vw_Fact_WorkOrder]
		WHERE
			BUKRS_CompanyCodeID = '5000'
			AND ActualCost > 0
	) AS SubQuery
	GROUP BY ILART_MaintenanceActivityTypeID
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
df_avg_cost_per_maint_activity = pd.read_sql_query(sql_query, conn)

# file_path = 'csv/average_cost_per_maint_activity_all.csv'
# df_avg_cost_per_maint_activity.to_csv(file_path, index=True)

# Display the DataFrame
df_avg_cost_per_maint_activity

C:\Users\r893859\AppData\Local\Temp\ipykernel_17160\985711224.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_avg_cost_per_maint_activity = pd.read_sql_query(sql_query, conn)


,ILART_MaintenanceActivityTypeID,AverageCost
0,C72,8646.802345
1,NGR,451.882170
2,C44,25206.133592
3,B16,2896.380000
4,XC1,67780.670000
...,...,...
169,P06,6812.133492
170,P16,1977.730107
171,E18,12507.830000
172,W01,1348.235000


In [2]:
import pandas as pd
import pyodbc

sql_query = """
    WITH work_order AS (
        SELECT
            AUFNR_Order,
            KTEXT_OrderDescr,
            TPLNR_FunctionalLocationInternalID,
            ILART_MaintenanceActivityTypeID,
            GSTRP_BasicStartDateID,
            GLTRP_BasicFinishDateID,
            ActualCost
        FROM [myANALYTICS_SP].[silver.fact.enterprise.asset].[vw_Fact_WorkOrder]
        WHERE
            BUKRS_CompanyCodeID = '5000'
            AND ActualCost > 0
    ),
    all_floc AS (
        SELECT
            FLOC_TPLNR_FunctionalLocationInternalKey,
            FLOC_STRNO_FunctionalLocation,
            FLOC_PLTXT_FunctionalLocationDescr,
            --FLOC_TYPTX_FunctionalLocationCategoryDescr,
            --FLOC_EQART_TechnicalObjectType,
            FLOC_EARTX_TechnicalObjectTypeDescr,
            --FLOC_BUKRS_CompanyCode,
            --FLOC_STORT_Location,
            FLOC_STORT_LocationDescr
        FROM [myANALYTICS_SP].[silver.dimension.enterprise.asset].[vw_Dim_FunctionalLocation]
        WHERE FLOC_BUKRS_CompanyCode = '5000'
    )

    SELECT
        work_order.AUFNR_Order,
        work_order.KTEXT_OrderDescr,
        work_order.ILART_MaintenanceActivityTypeID,
        work_order.GSTRP_BasicStartDateID,
        work_order.GLTRP_BasicFinishDateID,
        work_order.ActualCost,
        all_floc.FLOC_STRNO_FunctionalLocation,
        all_floc.FLOC_PLTXT_FunctionalLocationDescr,
        all_floc.FLOC_EARTX_TechnicalObjectTypeDescr,
        all_floc.FLOC_STORT_LocationDescr
    FROM work_order
        LEFT JOIN all_floc ON work_order.TPLNR_FunctionalLocationInternalID = all_floc.FLOC_TPLNR_FunctionalLocationInternalKey
    ORDER BY work_order.GSTRP_BasicStartDateID
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
df_all_work_orders = pd.read_sql_query(sql_query, conn)

# file_path = 'csv/work_orders_all.csv'
# df_all_work_orders.to_csv(file_path, index=True)

# Display the DataFrame
df_all_work_orders

C:\Users\r893859\AppData\Local\Temp\ipykernel_17160\1426430670.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_all_work_orders = pd.read_sql_query(sql_query, conn)


,AUFNR_Order,KTEXT_OrderDescr,ILART_MaintenanceActivityTypeID,GSTRP_BasicStartDateID,GLTRP_BasicFinishDateID,ActualCost,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr
0,50411005,FY22 Inventory Management BW Gladstone,A07,20210701,20220630,8716.56,None,None,None,None
1,30687481,EXTRA LOW VOLTAGE INSPECT 6 MONTH,P06,20210701,20210701,331.87,PN-SP-FL-CFL101,CFL101 FORKLIFT PRAMAC LIFTER,Plant: Forklift,None
2,80104553,NW_IV826_OH_Headspan_Elec Design,E13,20210701,20210701,4527.00,None,None,None,None
3,58164905,Traction Power Systems Site - F Svc,NSV,20210701,20210701,396.05,ST001610-01,BO 60.5 Km PSC,Substation,Bolingbroke
4,80104554,NW_IV826_OH_FW-Edungalba_Elec Dgn,RCP,20210701,20210701,35975.00,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
243497,600001393,Warren 46A C13,C17,20260701,20270630,0.00,TO000697,25.444KM 46A TURNOUT WARREN,Electrical Turnout,Warren
243498,600001392,Warren 45A C13,C17,20260701,20270630,0.00,TO000698,25.541KM 45A TURNOUT WARREN,Electrical Turnout,Warren
243499,30714563,XQ05RH RRV5605 HIRAIL BATTERY CHANGE 3YR,P06,20270301,20270301,0.00,PN-SP-HR-RR5605,RRV5605 ISUZU RRV TRUCK 7.5T XQ05RH,None,Blackwater Infrastructure Maintenance
243500,57006432,Ingsdon Dn P & C weld vee 101A,NRP,20300101,20300101,0.00,TO000362,11.251KM 101A TURNOUT INGSDON,Electrical Turnout,Ingsdon


In [3]:
df_test1 = df_all_work_orders[df_all_work_orders['ILART_MaintenanceActivityTypeID'] == 'C13']
df_test1 = df_test1[df_test1['ActualCost'] > 0]
df_test1

,AUFNR_Order,KTEXT_OrderDescr,ILART_MaintenanceActivityTypeID,GSTRP_BasicStartDateID,GLTRP_BasicFinishDateID,ActualCost,FLOC_STRNO_FunctionalLocation,FLOC_PLTXT_FunctionalLocationDescr,FLOC_EARTX_TechnicalObjectTypeDescr,FLOC_STORT_LocationDescr
1612,80104227,PAS UP 8cd SUPP BCD Bnd Int,C13,20210707,20210707,27143.48,TO000470-01,PS8C Pts (SNX),Electric Points,Praguelands
1653,80103945,Hay Point 10AB C13,C13,20210707,20210707,7808.41,TO000568,3.627KM 10A/B TURNOUT HAY POINT,Electrical Turnout,Hay Point
1697,80104147,Hay Point 10AB C13,C13,20210707,20210714,97416.58,TO000568,3.627KM 10A/B TURNOUT HAY POINT,Electrical Turnout,Hay Point
1769,80103949,Praguelands TO 8CD C13,C13,20210707,20210707,10276.31,TO000470,14.455KM 8C/D TURNOUT PRAGUELANDS,Electrical Turnout,Praguelands
1822,80104489,HAY 10ab SUPP BCD Bnd Int,C13,20210707,20210708,10072.33,TO000568-01,HP10A Pts,Electric Points,Hay Point
...,...,...,...,...,...,...,...,...,...,...
215892,80117047,Mt Miller Down - 10CD - C13,C13,20240416,20240416,3219.95,TO000574,542.213KM 10C/D TURNOUT MT MILLER,Electrical Turnout,Mount Miller
216106,80116613,Mt Larcom Down - 10AB - C13,C13,20240417,20240417,4829.93,TO000910,564.885KM 10A TURNOUT MT LARCOM,Electrical Turnout,Mount Larcom
221204,80116835,SR_BOND/MAINT_CLARKE 7A,C13,20240514,20240515,1076.54,TO000940,63.947KM 7A TURNOUT CLARKE,Electrical Turnout,Clarke
221205,80116836,SR_BOND/MAINT_STOWE 7A,C13,20240514,20240515,574.15,TO000958,26.305KM 7A TURNOUT STOWE,Electrical Turnout,Stowe


In [4]:
average_value = df_test1['ActualCost'].mean()
average_value

40705.479978494615

In [5]:
len(df_test1)

465